Hi all,
Next Monday (Feb 28th) we will hold the 4th coaching session about "Individual practice incorporating graphics into papers". As usual, you will be assigned to one breakout room with 5-6 fellow 1st-year Ph.D. students and 1 senior Ph.D. student as the coach. Please each bring 1-3 figures/plots/graphs about the potential final key result of your 599 project. It doesn't have to be a real result since most of us are not quite there yet. You can use fake data to create your own plot, or directly use available plots from the literature review.

The goal here is to use the plot to show the final output of your 599 project, which can be the ending part of your final presentation to the committee members.

In [6]:
# imports
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.simplefilter("ignore", UserWarning)
import nelpy as nel  
import nelpy.plotting as npl
import jagular as jag

In [7]:
prefix = '/home/kaylavokt/Desktop/Whrath/Day0/' # EDIT
rec = 'preFamSleep'
files = prefix + rec + '.rec' # EDIT
os.system("mkdir " + prefix + rec + "_channels")

# Tetrodes of use: 1, 3, 5, 7

#########################################################################
# jagular extraction parameters
jfm = jag.io.JagularFileMap(files) # jagular file map
max_gap_size = 10000 # maximum number of samples to interpolate
ts_out = prefix + rec + '_timestamps.raw' # output file, customize as needed
ch_out_prefix = prefix + rec + '_channels/'
subset = [0, 1, 2, 3, 8, 9, 10, 11, 16, 17, 18, 19, 24, 25, 26, 27] # which channels to extract
block_size = 2**21 # controls how much data to read at a time
verbose = False
###########################################################################

jag.utils.extract_channels(
    jfm=jfm,
    max_gap_size=max_gap_size,
    ts_out=ts_out,
    ch_out_prefix=ch_out_prefix,
    subset=subset,
    block_size=block_size,
    verbose=verbose
)

In [8]:
ts = np.fromfile(prefix + rec + '_timestamps.raw', dtype = np.uint32) # raw timestamps not in seconds
timestamps = (ts -ts[0])/30000

In [ ]:
# Create Analog Signal Array (ASA)
ch1 = np.fromfile(prefix + rec + "_channels/ch.00.raw", dtype = np.int16)

asa = nel.AnalogSignalArray(ch1,timestamps=timestamps,fs=3000)

In [11]:
# Detect Canonical Ripples
# Code from Josh's MS Defense

def extract_ripples(asa):
    eeg = nel.filtering.sosfiltfilt(asa, fl=150, fh=250)
    env = nel.utils.signal_envelope1D(eeg, sigma=0.004)
    bounds, _, _ = nel.utils.get_events_boundaries(env.data,
    PrimaryThreshold=env.mean() + 3*env.std(),
    SecondaryThreshold=env.mean(),
    minThresholdLength=0.015,
    minLength=0.05,
    ds=1/env.fs)
    timebounds = env.time[bounds]
    timesbounds[:, 1] += 1/env.fs
    return nel.EpochArray(timebounds)

# extract_ripples(asa)

TypeError: 'numpy.float64' object cannot be interpreted as an integer